In [8]:
include("nn_ops.jl")

NNOps

In [2]:
batch = 1
in_height = 6
in_width = 6
stride_height = 2
stride_width = 2
pooled_height = round(Int, in_height/stride_height, RoundUp)
pooled_width = round(Int, in_width/stride_width, RoundUp)
in_channels = 1
filter_height = 2
filter_width = 2
out_channels = 32
bigM = 10000

srand(5)
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)
filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)
x_conv_relu_maxpool_actual = NNOps.convlayer(x_conv_relu_actual, filter, (1, 2, 2, 1));
# x_conv_actual = NNOps.conv2d(x_actual, filter)
# x_conv_relu_actual = NNOps.relu(x_conv_actual)
# x_conv_relu_maxpool_actual = NNOps.maxpool(x_conv_relu_actual, (1, 2, 2, 1));

In [5]:
using JuMP
using Gurobi

m = Model(solver=GurobiSolver())

@variable(m, ve[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m, vx_conv[1:batch, 1:in_height, 1:in_width, 1:out_channels])
# @variable(m, vx_conv_relu[1:batch, 1:in_height, 1:in_width, 1:out_channels])
# @variable(m, vx_conv_relu_maxpool[1:batch, 1:pooled_height, 1:pooled_width, 1:out_channels])
@constraint(m, vx .== x_current) # input

# 1. Only using convolution constraint
# vx_conv = NNOps.conv2dconstraint(m, vx+ve, filter)
# @constraint(m, conv2d(vx+ve, filter) .== vx_conv)
# @constraint(m, vx_conv .== x_conv_actual)

# 2. Adding relu layer
# vx_conv_relu = NNOps.reluconstraint(m, vx_conv, 10000)
# @constraint(m, vx_conv_relu .== x_conv_relu_actual)

# 3. Adding maxpool layer
# vx_conv_relu_maxpool = NNOps.maxpoolconstraint(m, vx_conv_relu, (2, 2), 10000)
# @constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual)

# 1-3a. Directly adding constraints
vx_conv_relu_maxpool = NNOps.convlayerconstraint(m, vx+ve, filter, (2, 2), 10000)
@constraint(m, vx_conv_relu_maxpool .== x_conv_relu_maxpool_actual)


@objective(m, Min, sum(ve.^2))

:Min

In [6]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(ve))

Optimize a model with 8676 rows, 6120 columns and 25348 nonzeros
Model has 36 quadratic objective terms
Variable types: 3816 continuous, 2304 integer (2304 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 2980 rows and 1828 columns
Presolve time: 0.03s
Presolved: 5696 rows, 4292 columns, 16160 nonzeros
Presolved model has 36 quadratic objective terms
Variable types: 1988 continuous, 2304 integer (2304 binary)
Found heuristic solution: objective 7905.6364327
Found heuristic solution: objective 5153.7274699

Root relaxation: objective 1.813530e+01, 6774 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   18.13530    0 1040 5153.72747   18.13530   100%     -    0s
     0     0   18.13

In [ ]:
x_conv_current = NNOps.conv2d(x_current+getvalue(ve), filter)
x_conv_relu_current = NNOps.relu(x_conv_current)
x_conv_relu_maxpool_current = NNOps.maxpool(x_conv_relu_current, (1, 2, 2, 1));

In [ ]:
maximum(abs(x_conv_current - x_conv_actual))

In [ ]:
maximum(abs(x_conv_relu_current - x_conv_relu_actual))

In [ ]:
maximum(abs(x_conv_relu_maxpool_current - x_conv_relu_maxpool_actual))

In [ ]:
maximum(abs(x_conv_actual))

In [ ]:
maximum(abs(x_conv_relu_actual))

In [4]:
x_conv_relu_maxpool_actual[:]

288-element Array{Int64,1}:
 144
 165
 119
 343
 386
 258
 352
 254
 309
 144
 165
 119
 343
   ⋮
 352
 254
 309
 144
 165
 119
 343
 386
 258
 352
 254
 309